In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [18]:
from matplotlib import rcParams
rcParams["figure.figsize"] = 20, 6
sns.set(font_scale=1.5)

In [19]:
DF_PATH = os.getcwd() + "/csv/used_cars.csv"
df = pd.read_csv(DF_PATH, sep=";", encoding = "ISO-8859-1")
df.head()

,name,year,mileage,engine_power_kW,engine_power_hp,fuel,consumption,ecology,gearbox,price,currency,country,capacity,other,engine_size,nr_of_accidents,color_exterior,color_interior
0,Daewoo Rezzo,2003,200000 km,89 kW,121 hp,Gasoline,9 l/100 km,236 g CO2/km,Manual,"199,00",EUR,Germany,NaN,NaN,NaN,NaN,NaN,NaN
1,Volkswagen Golf,1994,222275 km,55 kW,75 hp,Gasoline,0 l/100 km,0 g CO2/km,Manual,"200,00",EUR,Germany,NaN,NaN,NaN,NaN,NaN,NaN
2,Ford Fiesta,2001,153000 km,44 kW,60 hp,Gasoline,7.3 l/100 km,171 g CO2/km,Manual,"290,00",EUR,Germany,NaN,NaN,NaN,NaN,NaN,NaN
3,Volkswagen Polo,1999,154000 km,44 kW,60 hp,Gasoline,6.3 l/100 km,151 g CO2/km,Manual,"350,00",EUR,Germany,NaN,NaN,NaN,NaN,NaN,NaN
4,Nissan Micra,1998,140000 km,40 kW,54 hp,Gasoline,6 l/100 km,152 g CO2/km,Manual,"380,00",EUR,Germany,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.shape

(1816, 18)

Lowercase cars' `name`.

In [21]:
df.name = df.name.str.lower()

Fill missing years with 0 and convert to int.

In [22]:
df.loc[df.year == "-", ["year"]] = 0
df.loc[df.year == '0', ["year"]] = 0

In [23]:
df.year = df.year.astype(int)

Substitute missing values for `ecology` and `consumption`  with nan values.

In [24]:
df = df[df.engine_power_kW != "#NAZWA?"]
df.loc[df.ecology == '-/- (CO2/km)', ["ecology"]] = np.nan
df.loc[df.consumption == '-/- (l/100 km)', ["consumption"]] = np.nan

Convert `engine_power_kW`, `engine_power_hp`, `consumption` and `ecology` to float.

Rename columns.

In [25]:
df.engine_power_kW = df.engine_power_kW.str.replace(" kW", "", regex=False).astype(float)
df.engine_power_hp = df.engine_power_hp.str.replace(" hp", "", regex=False).astype(float)
df.ecology = df.ecology.str.replace(" g CO2/km", "", regex=False).astype(float)
df.consumption = df.consumption.str.replace(" l/100 km", "", regex=False).astype(float)
df.rename(columns={"ecology": "ecology_(gCO2/km)", "consumption":"consumption_(l/100 km)", "fuel":"fuel_type"}, inplace=True)

Convert miles in `mileage` to km. Convert `mileage` to float.

In [26]:
miles_to_km = 1.609344

miles_df = df[df.mileage.str.contains("mi")].mileage.str.replace(r"[a-z]+", "")
miles_df = round(miles_df.str.replace(',', "", regex=False).astype(float) * miles_to_km, 0)

km_df = df[~df.mileage.str.contains("mi")].mileage.str.replace(r"[a-z]+", "")
km_df = km_df.str.replace(',', "", regex=False).astype(float)

contains_miles = df.mileage.str.contains("mi")
df.loc[contains_miles, ['mileage']] = miles_df

contains_miles = ~contains_miles
df.loc[contains_miles, ['mileage']] = km_df

Convert all prices from original currencies to EUR.

In [27]:
from currency_converter import CurrencyConverter
c = CurrencyConverter()
currencies = df.currency.unique()
# currencies = [c for c in currencies if c != "EUR"]

df.price = df.price.str.replace(",", ".").astype(float)

for currency in currencies:
    price = df[df.currency == currency].price
    curr = [round(c.convert(p, currency, "EUR"), 2) for p in (list(price))]
    df.loc[df.currency == currency, "price_eur"] = curr

# df = df.drop(columns=["price", "currency"])

Combine `capacity` and `engine_size` into one column, clean and convert to int.

In [28]:
df.engine_size = df.engine_size.combine_first(df.capacity)

In [29]:
df.engine_size = df.engine_size.str.replace(" cc", "", regex=False)
df.engine_size = df.engine_size.str.replace(" cm33", "", regex=False)
df.engine_size = df.engine_size.str.replace(",", "", regex=False)
df.engine_size = df.engine_size.fillna(value=0)
df.engine_size = df.engine_size.astype(float)

In [30]:
df.drop(columns=["capacity", 'other'], inplace=True)

Change NaN in object columns to `"unknown"`.

In [31]:
df.fuel_type.fillna("unknown", inplace=True)
df.gearbox.fillna("unknown", inplace=True)
df.color_exterior.fillna("unknown", inplace=True)
df.color_interior.fillna("unknown", inplace=True)

In [32]:
df.dtypes

name                       object
year                        int32
mileage                    object
engine_power_kW           float64
engine_power_hp           float64
fuel_type                  object
consumption_(l/100 km)    float64
ecology_(gCO2/km)         float64
gearbox                    object
price                     float64
currency                   object
country                    object
engine_size               float64
nr_of_accidents           float64
color_exterior             object
color_interior             object
price_eur                 float64
dtype: object

Add, analyze and clean column `mark`.

In [33]:
marks = []
for i in range(len(df)):
    name = df.iloc[i][0]
    name = name.lower().replace(".", "").split(" ")
    for word in name:
        if not word.isdigit():
            marks.append(word)
            break

In [34]:
df.insert(1, 'mark', marks)

In [35]:
df.loc[df.mark.str.contains("oda"), 'mark'] = "skoda"
df.loc[df.mark.str.contains("skodia"), 'mark'] = "skoda"
df.loc[df.mark.str.contains("citr"), 'mark'] = "citroen"
df.loc[df.mark.str.contains("mercedes-benz"), 'mark'] = "mercedes"
df.loc[df.mark.str.contains("vw"), 'mark'] = "volkswagen"
df.loc[df.mark=="late", 'mark'] = "nissan"
df.loc[df.mark.str.contains("cooper") & df.name.str.contains("mini"), 'mark'] = "cooper"
df.loc[df.mark == "land", 'mark'] = "rover"
df.loc[df.mark == "door", 'mark'] = "rover"
# df.loc[df.mark.str.contains("vw"), 'mark'] = "volkswagen"

Get rid of vehicles of obscure title and non-cars.

In [36]:
incorrect_vals = ['very', 'low', 'green', 'lag', 'new', 'case', 'wiola',
                 'reluctant', 'indian', 'tmt', 'hamm', 'atlas', 'man', 'months']
non_car_manufacturer = ['jcb', 'kverneland', 'goldhofer', 'laverda', 'kobelco',
                        'bomag', 'fendt', 'merlo', 'kubota', 'bobcat', 'manitou',
                        'komatsu', 'caterpillar', 'terex', 'takeuchi', 'john', 'wacker',
                        'case', 'carnehl', 'vaderstad', 'skuter', 'schaffer', 'iveco']

df = df[~df.mark.isin(incorrect_vals)]
df = df[~df.mark.isin(non_car_manufacturer)]

In [37]:
df.head()

,name,mark,year,mileage,engine_power_kW,engine_power_hp,fuel_type,consumption_(l/100 km),ecology_(gCO2/km),gearbox,price,currency,country,engine_size,nr_of_accidents,color_exterior,color_interior,price_eur
0,daewoo rezzo,daewoo,2003,200000,89.0,121.0,Gasoline,9.0,236.0,Manual,199.0,EUR,Germany,0.0,NaN,unknown,unknown,199.0
1,volkswagen golf,volkswagen,1994,222275,55.0,75.0,Gasoline,0.0,0.0,Manual,200.0,EUR,Germany,0.0,NaN,unknown,unknown,200.0
2,ford fiesta,ford,2001,153000,44.0,60.0,Gasoline,7.3,171.0,Manual,290.0,EUR,Germany,0.0,NaN,unknown,unknown,290.0
3,volkswagen polo,volkswagen,1999,154000,44.0,60.0,Gasoline,6.3,151.0,Manual,350.0,EUR,Germany,0.0,NaN,unknown,unknown,350.0
4,nissan micra,nissan,1998,140000,40.0,54.0,Gasoline,6.0,152.0,Manual,380.0,EUR,Germany,0.0,NaN,unknown,unknown,380.0


In [38]:
# df.loc[df.mark == "smart"]["name"]

In [39]:
df.shape

(1700, 18)

In [40]:
df.mark.unique()

array(['daewoo', 'volkswagen', 'ford', 'nissan', 'opel', 'mitsubishi',
       'renault', 'audi', 'fiat', 'bmw', 'saab', 'skoda', 'mercedes',
       'lancia', 'seat', 'toyota', 'volvo', 'peugeot', 'mazda', 'citroen',
       'kia', 'rover', 'chevrolet', 'smart', 'mini', 'hyundai', 'dacia',
       'suzuki', 'honda', 'daihatsu', 'alfa', 'porsche', 'bentley',
       'maserati', 'lexus', 'jeep', 'maybach', 'infiniti', 'triumph',
       'vã¤derstad', 'chrysler', 'doosan', 'massey', 'claas', 'mccormick',
       'hitachi', 'lamborghini', 'ssangyong', 'mg', 'jaguar', 'kymco',
       'stokota', 'yamaha', 'cf', 'can-am', 'vauxhall', 'mgtf', 'audt',
       'aston', 'dodge', 'ds', 'acura', 'ram', 'gmc', 'ferrari',
       'cadillac', 'lincoln', 'hummer', 'pontiac', 'scion', 'buick',
       'subaru'], dtype=object)

In [41]:
df.mark.value_counts().to_frame()

,mark
mercedes,197
ford,194
nissan,123
toyota,104
volkswagen,98
...,...
cf,1
lamborghini,1
buick,1
scion,1


In [42]:
df.to_csv("csv/new_csv/cleaned_data.csv", index=False)